# 🎯 03 - Model Eğitimi ve Değerlendirme

Bu defterde:
- Özellik mühendisliği ile hazırlanan veri üzerinden model eğitilecek,
- Farklı algoritmalarla denemeler yapılacak,
- Model başarımı değerlendirilecek,
- En iyi model ile test seti tahmin edilecek.

Amaç: MAE (Mean Absolute Error) değerini düşürmek ve test verisi için anlamlı tahminler elde etmek.


In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
sys.path.append(os.path.abspath("../src"))  # src klasörünü yol olarak ekle

from model_utils import evaluate_model
# İşlenmiş veriyi oku
df = pd.read_csv("../data/processed_df.csv", parse_dates=["tarih"])

# Kontrol
df.head()


,tarih,ilce,bildirimsiz_sum,bildirimli_sum,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,...,yil,ay,gun,hafta,haftanin_gunu,lag_1,lag_2,lag_3,rolling_mean_3,rolling_mean_7
0,2021-01-01,izmir-aliaga,5.0,0,13.275000,55.045833,73.020833,85.991667,167.204167,4.266667,...,2021,1,1,53,4,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,izmir-aliaga,13.0,0,12.870833,36.125000,92.858333,82.137500,106.650000,2.891667,...,2021,1,2,53,5,5.0,NaN,NaN,5.000000,5.000000
2,2021-01-03,izmir-aliaga,4.0,0,12.633333,64.870833,39.266667,79.120833,122.520833,3.437500,...,2021,1,3,53,6,13.0,5.0,NaN,9.000000,9.000000
3,2021-01-04,izmir-aliaga,9.0,0,13.733333,47.783333,78.945833,70.312500,123.408333,4.608333,...,2021,1,4,1,0,4.0,13.0,5.0,7.333333,7.333333
4,2021-01-05,izmir-aliaga,2.0,0,14.375000,30.379167,95.716667,75.220833,195.720833,3.991667,...,2021,1,5,1,1,9.0,4.0,13.0,8.666667,7.750000


In [29]:
# Özellik isimlerindeki LightGBM'e sorun çıkaracak karakterleri düzelt
df.columns = df.columns.str.replace("[:.]", "_", regex=True)


In [30]:
# processed_df.csv'den ayrım
train_df = df[df["bildirimsiz_sum"].notnull()].copy()
test_df = df[df["bildirimsiz_sum"].isnull()].copy()


In [31]:
# Hedef ve girdi kolonları
target = "bildirimsiz_sum"

# Modele sokmak istemediğimiz sütunlar
drop_cols = ["bildirimsiz_sum", "tarih", "ilce"]

feature_cols = [col for col in train_df.columns if col not in drop_cols]

print("Kullanılan özellik sayısı:", len(feature_cols))


Kullanılan özellik sayısı: 20


In [32]:
# Tarihe göre sırala
train_df = train_df.sort_values("tarih")

# %80 eğitim, %20 validasyon
split_index = int(len(train_df) * 0.8)
train_data = train_df.iloc[:split_index]
val_data = train_df.iloc[split_index:]

# X ve y
X_train = train_data[feature_cols]
y_train = train_data[target]
X_val = val_data[feature_cols]
y_val = val_data[target]

print("Zaman temelli eğitim boyutu:", X_train.shape)
print("Zaman temelli doğrulama boyutu:", X_val.shape)



Zaman temelli eğitim boyutu: (38518, 20)
Zaman temelli doğrulama boyutu: (9630, 20)


In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=100, random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42)
}


In [34]:
X_train_clean = X_train.dropna()
y_train_clean = y_train.loc[X_train_clean.index]

In [ ]:
results = {}

for name, model in models.items():
    print(f"\n🔍 {name} modeli değerlendiriliyor...")
    mae = evaluate_model(model, X_train_clean, y_train_clean, n_splits=5)
    print(f"Ortalama MAE: {mae:.4f}")
    results[name] = mae



🔍 LinearRegression modeli değerlendiriliyor...
📉 Ortalama MAE: 2.9132

🔍 RandomForest modeli değerlendiriliyor...
📉 Ortalama MAE: 3.1165

🔍 XGBoost modeli değerlendiriliyor...
📉 Ortalama MAE: 3.2505

🔍 LightGBM modeli değerlendiriliyor...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2465
[LightGBM] [Info] Number of data points in the train set: 6220, number of used features: 19
[LightGBM] [Info] Start training from score 6.831833
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2518
[LightGBM] [Info] Number of data points in the train set: 12435, number of used features: 19
[L

In [36]:
# X ve y (NaN içermeyen)
X_full = train_df[feature_cols].dropna()
y_full = train_df.loc[X_full.index, target]

# Test verisi
X_test = test_df[feature_cols]


In [38]:
models = {    
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=100, random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42)
}

In [ ]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

# Örnek submission dosyasını yükle (sıra ve format için)
base_submission = pd.read_csv("../data/sample_submission.csv")

# Tahmin sonuçlarını saklamak istersen
results = {}

# Tüm modelleri sırayla eğitip tahmin et
for name, model in models.items():
    print(f"\n{name} modeli ile tüm veride eğitim başlıyor...")
    
    # Model eğitimi
    model.fit(X_full, y_full)
    
    # Tahmin
    preds = model.predict(X_test)
    
    # Submission dosyası oluştur
    submission_df = base_submission.copy()
    submission_df["bildirimsiz_sum"] = preds.round().astype(int)
    
    # Kaydet
    output_path = f"../submission/submission_{name}.csv"
    submission_df.to_csv(output_path, index=False)
    print(f"Tahminler '{output_path}' dosyasına kaydedildi.")



🚀 RandomForest modeli ile tüm veride eğitim başlıyor...
📁 Tahminler '../submission/submission_RandomForest.csv' dosyasına kaydedildi.

🚀 XGBoost modeli ile tüm veride eğitim başlıyor...
📁 Tahminler '../submission/submission_XGBoost.csv' dosyasına kaydedildi.

🚀 LightGBM modeli ile tüm veride eğitim başlıyor...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2674
[LightGBM] [Info] Number of data points in the train set: 46792, number of used features: 20
[LightGBM] [Info] Start training from score 5.896414
📁 Tahminler '../submission/submission_LightGBM.csv' dosyasına kaydedildi.

🚀 CatBoost modeli ile tüm veride eğitim başlıyor...
📁 Tahminler '../submission/submission_CatBoost.csv' dosyasına kaydedildi.
